In [ ]:
#pip install -U transformers==4.35.0 sentence-transformers==2.2.2 chromadb==0.4.6 accelerate

# Creación de un Entorno de Python

En este documento, vamos a explicar cómo configurar un nuevo entorno de Python utilizando tanto `venv` como `conda`.

## Usando venv

`venv` es una herramienta que viene preinstalada con Python (versión 3.3 o superior) y permite crear entornos virtuales de Python aislados.

### Pasos

1. **Crear un entorno virtual:** Abre la terminal y ejecuta el siguiente comando en el directorio donde desees crear el entorno virtual.

   ```bash
   python3 -m venv nombre_del_entorno


2. **Activar el entorno virtual:** Utiliza el siguiente comando para activar el entorno.


  Linux/Mac:

```bash
    source nombre_del_entorno/bin/activate
```


   Windows:

```bash
    .\nombre_del_entorno\Scripts\activate
```


3. **Instalar dependencias desde requirements.txt con pip:** Una vez activado el entorno, puedes instalar todas las dependencias necesarias desde un archivo `requirements.txt` utilizando:

   ```bash
   pip install -r requirements.txt

4. **Desactivar el entorno virtual:** Para salir del entorno virtual, simplemente ejecuta:

```bash
   deactivate

## Usando Conda
conda es un sistema de gestión de paquetes y de entorno que puede instalar paquetes de diferentes lenguajes.

### Pasos
1. **Instalar Conda:** Puedes descargar e instalar Conda desde [este enlace](https://www.anaconda.com/download).

2. **Crear un entorno Conda:** Para crear un nuevo entorno con Conda, abre la terminal y ejecuta:

```bash
    conda create --name nombre_del_entorno python=3.9 ipykernel
```

Para esta biblioteca, vamos a usar python 3.9 y para interactuar con Jupyter Notebooks vamos a usar "ipykernel" o "jupyter"

3. **Activar el entorno Conda:** Utiliza el siguiente comando para activar el entorno.

```bash
    conda activate nombre_del_entorno
```
4. **Instalar dependencias desde requirements.txt con pip:** Al igual que con venv, puedes instalar las dependencias necesarias desde un archivo requirements.txt en el entorno Conda activado utilizando:
```bash
    pip install -r requirements.txt
```

5. Desactivar el entorno Conda: Para salir del entorno Conda, simplemente ejecuta:

```bash
    conda deactivate
```

In [2]:
# 1️⃣ Importar librerías
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import os
import numpy as np
import uuid
import requests
import json

# Modelo de embeddings
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model = SentenceTransformer(model_name)

def get_embeddings(text):
    return model.encode(text).tolist()

# Inicializar ChromaDB con persistencia
client = chromadb.PersistentClient(path="./chroma_store")
collection = client.get_or_create_collection(name="documentos_ia")

def cargar_documentos():
    docs_path = "docs/"
    documentos = []
    metadatos = []
    ids = []

    for filename in os.listdir(docs_path):
        if filename.endswith(".txt"):
            with open(os.path.join(docs_path, filename), 'r', encoding="utf-8") as f:
                text = f.read().strip()
                chunks = [text[i:i+500] for i in range(0, len(text), 500)]
                
                for i, chunk in enumerate(chunks):
                    documentos.append(chunk)
                    metadatos.append({"source": filename})
                    ids.append(str(uuid.uuid4()))  # ID único
    
    return documentos, metadatos, ids

def crear_base_datos():
    docs, metas, ids = cargar_documentos()
    embeddings = [get_embeddings(d) for d in docs]
    collection.add(documents=docs, embeddings=embeddings, metadatas=metas, ids=ids)
    print("✅ Base de datos creada con éxito.")

# --- CRUD FUNCIONES ---

def create_example(new_doc):
    """Añadir un documento nuevo"""
    embedding = get_embeddings(new_doc)
    doc_id = str(uuid.uuid4())
    metadata = {"source": "nuevo_documento.txt"}
    
    collection.add(
        documents=[new_doc],
        embeddings=[embedding],
        metadatas=[metadata],
        ids=[doc_id]
    )
    print(f"✅ Documento añadido con ID: {doc_id}")

def read_example(query):
    """Consultar documentos similares"""
    query_emb = get_embeddings(query)
    results = collection.query(
        query_embeddings=[query_emb],
        n_results=1
    )
    
    if results["documents"]:
        print(f"📄 Mejor coincidencia: {results['documents'][0][0]}")
        print(f"📁 Fuente: {results['metadatas'][0][0]['source']}")
        print(f"🆔 ID: {results['ids'][0][0]}")
    else:
        print("⚠️ No se encontraron resultados.")

def get_documents_by_id(doc_id):
    """Obtener un documento por ID"""
    results = collection.get(ids=[doc_id], include=["documents", "metadatas", "embeddings"])
    
    if results["documents"]:
        print(f"📄 Documento: {results['documents'][0]}")
        print(f"📁 Fuente: {results['metadatas'][0]['source']}")
    else:
        print("⚠️ Documento no encontrado.")

def update_example(old_id, new_text):
    """Actualizar un documento (eliminar y reinsertar)"""
    collection.delete(ids=[old_id])
    
    new_id = str(uuid.uuid4())
    embedding = get_embeddings(new_text)
    metadata = {"source": "documento_actualizado.txt"}
    
    collection.add(
        documents=[new_text],
        embeddings=[embedding],
        metadatas=[metadata],
        ids=[new_id]
    )
    print(f"♻️ Documento actualizado. Nuevo ID: {new_id}")

def delete_example(doc_id):
    """Eliminar por ID"""
    collection.delete(ids=[doc_id])
    print(f"🗑️ Documento con ID '{doc_id}' eliminado.")

In [3]:
crear_base_datos()

✅ Base de datos creada con éxito.


In [4]:
get_embeddings("Reynaldo González es un ingeniero de software con 10 años de experiencia en desarrollo web y móvil.")

[-0.09879041463136673,
 0.011900096200406551,
 -0.07934798300266266,
 -0.07173146307468414,
 0.04849894717335701,
 -0.08828753232955933,
 0.04442284256219864,
 0.06878751516342163,
 -0.04471663013100624,
 -0.019074415788054466,
 0.009390958584845066,
 0.0556383840739727,
 0.02038569562137127,
 0.005857520736753941,
 0.0571175254881382,
 0.03409785404801369,
 -0.033253442496061325,
 0.008881214074790478,
 0.022362586110830307,
 -0.06751402467489243,
 0.11720024794340134,
 -0.029926331713795662,
 0.0018126838840544224,
 -0.00767976138740778,
 -0.014972866512835026,
 0.0011125649325549603,
 0.006905629299581051,
 -0.016392141580581665,
 -0.025348829105496407,
 -0.09709931910037994,
 0.0348370224237442,
 0.06626584380865097,
 0.07979563623666763,
 0.050580158829689026,
 -0.024885069578886032,
 -0.00418727146461606,
 0.021157681941986084,
 -0.05807731673121452,
 -0.0793749988079071,
 0.012050828896462917,
 -0.1423081010580063,
 0.02995726279914379,
 -0.0037484760396182537,
 -0.0742330551147

In [5]:
results = collection.get(include=["embeddings", "documents", "metadatas"])

In [6]:
for id, content, metadata, embedding in zip(results["ids"], results["documents"], results["metadatas"], results["embeddings"]):
    print(f"ID: {id}, Contenido: {content[:100]} ", f"Metadata: {metadata}", f"Embedding: {embedding[:5]}...")

ID: 1c52b4c4-c49f-479a-a28a-0fa4f4c0b8c8, Contenido: El sol se alzaba sobre las colinas de Buenos Aires, pintando el cielo con tonos naranja y rosa. El a  Metadata: {'source': 'doc1.txt'} Embedding: [ 0.05201021 -0.04184367  0.03102785  0.03516811  0.00390171]...
ID: 7052b785-9349-48ba-901b-7792b64fa052, Contenido: ciosa continuaba, con el tráfico constante y el murmullo de las conversaciones. Los turistas se mara  Metadata: {'source': 'doc1.txt'} Embedding: [ 0.06561406  0.0027415   0.01852138 -0.00384713 -0.08065528]...
ID: 303974be-5fb5-4590-8bc2-8e879f1c6e0a, Contenido:  los restaurantes servían deliciosas parrilladas.

Hacia el norte, el elegante barrio de Recoleta in  Metadata: {'source': 'doc1.txt'} Embedding: [ 0.06329251 -0.057605    0.01019359  0.01427127 -0.11836304]...
ID: a9d153e1-4b79-4d80-86ac-d915d108f351, Contenido: y novelas exploran la realidad, la fantasía y la condición humana con una prosa exquisita.

El fútbo  Metadata: {'source': 'doc1.txt'} Embedding: [ 0.10750

In [ ]:
while True:
    opcion = input("Elige una opción o escribe ayuda: ")
    
    if opcion=="ayuda":
        print("\n--- Menú CRUD ---")
        print("1. Crear documento")
        print("2. Consultar por tema")
        print("3. Actualizar documento (por ID)")
        print("4. Eliminar documento (por ID)")
        print("5. Buscar documento (por ID)")
        print("6. Salir")

    if opcion == "1":
        nuevo_doc = input("Ingresa el texto a añadir: ")
        create_example(nuevo_doc)
        
    elif opcion == "2":
        consulta = input("Consulta: ")
        read_example(consulta)
        
    elif opcion == "3":
        doc_id = input("ID del documento a actualizar: ")
        nuevo_texto = input("Nuevo contenido: ")
        update_example(doc_id, nuevo_texto)
        
    elif opcion == "4":
        doc_id = input("ID del documento a eliminar: ")
        delete_example(doc_id)  

    elif opcion == "5":
        doc_id = input("ID del documento a buscar: ")
        get_documents_by_id(doc_id)
        
    elif opcion == "6":
        print("👋 Saliendo del programa.")
        break

✅ Documento añadido con ID: 2406af4a-9923-4639-822f-8508416aa634
📄 Mejor coincidencia: caudal y la belleza de su entorno selvático atraen a miles de visitantes cada año.

La Antártida Argentina es un territorio reclamado por el país, donde se realizan importantes investigaciones científicas en diversas áreas, desde la glaciología hasta la biología marina.

La astronomía tiene un lugar destacado en Argentina, con observatorios de renombre internacional como el Observatorio Pierre Auger, dedicado al estudio de los rayos cósmicos de ultra alta energía.

La biotecnología es un sector 
📁 Fuente: doc3.txt
🆔 ID: 2b357267-961b-4a4d-84bd-ff1ea98feadf
✅ Documento añadido con ID: bd7232fe-535c-4e2b-9093-3f156a504da7
♻️ Documento actualizado. Nuevo ID: 6f44a060-cebc-4b72-af6b-d05b68cf0c5a
🗑️ Documento con ID 'bd7232fe-535c-4e2b-9093-3f156a504da7' eliminado.
♻️ Documento actualizado. Nuevo ID: db4ea385-0b3a-4cb8-be6c-3832ed22b6e9
🗑️ Documento con ID 'no_existe' eliminado.
✅ Documento añadido con ID

In [ ]:
total_documentos = collection.count()
print(f"📊 Total de documentos en la colección: {total_documentos}")

**Ejercicio 1: Crear y Consultar**
¿Qué lección aprendemos sobre los requisitos de ChromaDB? 
 
Ejercicio opcional: Retrieval-Augmented Generation 
 
Intente levantar un llm en un endpoint local con ollama o LMStudio. Utilice el llm para 
generar respuestas aumentadas con contexto similar a una consulta dada. 


**Ejercicio 2: Actualizar y Eliminar**

Crea un documento con el texto: "Redes neuronales imitan procesos cerebrales". 
Encuentra su ID, luego actualízalo por: "Deep Learning es un subcampo de IA con redes neuronales profundas". 
Elimina el documento original usando el mismo ID. 

¿Qué ocurre si intentas actualizar un documento eliminado? 
¿Cómo se garantiza la integridad de los datos en las operaciones?

**Ejercicio 3: Errores y Validación**

Intenta eliminar un documento con el ID "no_existe". 
Crea un nuevo documento sin usar get_embeddings() (ejemplo: enviar un array vacío). 

¿Qué errores ocurren en cada caso? 
¿Cómo prevenir estos problemas?

**Ejercicio 4: Consultas por Tema**

Crea tres documentos: 
"Python es ideal para ciencia de datos" 
"JavaScript maneja interacciones web dinámicas" 
"Java se usa en aplicaciones empresariales" 
Realiza una consulta por la frase: "Lenguajes backend y frontend". 

¿Cuáles documentos aparecen como coincidencias? 
Analiza cómo las palabras clave influyen en los resultados.

**Ejercicio 5: Eliminaciones Selectivas**

Crea dos documentos con tópicos opuestos (ejemplo: uno sobre "Ecología" y otro sobre "Minería de datos"). 
Elimina el documento sobre ecología usando su ID. 
Verifica que solo quede un documento. 

¿Cómo usar collection.count() para confirmar?

**Ejercicio 6: Actualización en Cadena**

Crea un documento con el texto "IA y ética son temas actuales". 
Actualízalo dos veces: 
Primera actualización: "Ética en IA aborda dilemas morales" 
Segunda actualización: "El uso de IA debe estar regulado por principios éticos". 

¿Cómo sigue evolucionando el contenido del documento con cada update?

**Ejercicio 7: Consultas Ambiguas**

Crea dos documentos similares pero no idénticos: 
"Blockchain garantiza transacciones seguras" 
"Criptomonedas usan tecnología blockchain". 
Realiza una consulta por "Seguridad en finanzas digitales". 

¿Ambos documentos aparecen? Explica la similitud semántica.

**Ejercicio 8: Reiniciar Base de Datos**

Elimina todos los documentos usando delete() con el parámetro correcto. 
Vuelve a cargar la base desde cero con crear_base_datos(). 

¿Cuántos documentos hay ahora? (Usa collection.count()).

**Ejercicio 9: Errores de Inserción**

Intenta crear un documento sin texto (ejemplo: cadena vacía). 
Corrige el error usando la función get_embeddings(). 

¿Qué lección aprendemos sobre los requisitos de ChromaDB?

**Ejercicio opcional: Retrieval-Augmented Generation**

Intente levantar un llm en un endpoint local con ollama o LMStudio. Utilice el llm para generar respuestas aumentadas con contexto similar a una consulta dada.

## ¿Cómo sigue esto?

### de forma conceptual debemos:
1. Recuperar el contexto relevante con una busqueda como read_example(query) o similar.
2. Formular el prompt para el llm, juntando la consulta original con los embeddings relevantes.
3. Enviar la Petición al Endpoint (en este caso LM Studio).
4. Recibir y mostrar la respuesta.

In [ ]:
# Esta sección es para la integración con LM Studio y no es parte del CRUD. Es solo un ejemplo de cómo se podría hacer una consulta a un modelo de lenguaje como Llama 3 o similar.
# 2️⃣ Integración con LM Studio (Llama 3 o similar)
import requests
import json

LM_STUDIO_URL = "http://192.168.0.3:1234/v1/chat/completions"  

def query_llama(prompt):
    """Envia un prompt a LM Studio y recibe la respuesta."""
    headers = {
        "Content-Type": "application/json"
    }
    data = {
        "messages": [{"role": "user", "content": prompt}],
        "max_tokens": 256, 
        # Es posible agregar otros parámetros de generación aca (temperature, top_p, etc.)
    }
    try:
        response = requests.post(LM_STUDIO_URL, headers=headers, data=json.dumps(data))
        response.raise_for_status()
        return response.json()["choices"][0]["message"]["content"]
    except requests.exceptions.RequestException as e:
        print(f"Error al conectar con LM Studio: {e}")
        return None
    except (KeyError, json.JSONDecodeError) as e:
        print(f"Error al procesar la respuesta de LM Studio: {e}")
        return None

def responder_pregunta(pregunta):
    """Busca documentos relevantes y luego consulta a Llama."""
    query_emb = get_embeddings(pregunta)
    results = collection.query(
        query_embeddings=[query_emb],
        n_results=3  # Obtener algunos documentos relevantes como contexto
    )

    if results["documents"]:
        contexto = "\n\n".join(results["documents"][0])
        prompt = f"Basándote en la siguiente información:\n\n{contexto}\n\nResponde a la siguiente pregunta: {pregunta}" # Juntar el contexto obtenido y la pregunta
        respuesta = query_llama(prompt)
        return respuesta
    else:
        return "No se encontraron documentos relevantes para responder a tu pregunta."

# --- Ejemplo de uso ---
if __name__ == "__main__":
    crear_base_datos() 

    while True:
        pregunta_usuario = input("Pregúntame algo sobre los documentos (o escribe 'salir'): ")
        if pregunta_usuario.lower() == 'salir':
            break

        respuesta_ia = responder_pregunta(pregunta_usuario)
        if respuesta_ia:
            print(f"🤖 Respuesta de la IA: {respuesta_ia}")

✅ Base de datos creada con éxito.
Error al conectar con Ollama: 404 Client Error: Not Found for url: http://localhost:11434/api/v1/generate
Error al conectar con Ollama: 404 Client Error: Not Found for url: http://localhost:11434/api/v1/generate
Error al conectar con Ollama: 404 Client Error: Not Found for url: http://localhost:11434/api/v1/generate
Error al conectar con Ollama: 404 Client Error: Not Found for url: http://localhost:11434/api/v1/generate
Error al conectar con Ollama: 404 Client Error: Not Found for url: http://localhost:11434/api/v1/generate
Error al conectar con Ollama: 404 Client Error: Not Found for url: http://localhost:11434/api/v1/generate
Error al conectar con Ollama: 404 Client Error: Not Found for url: http://localhost:11434/api/v1/generate
Error al conectar con Ollama: 404 Client Error: Not Found for url: http://localhost:11434/api/v1/generate
Error al conectar con Ollama: 404 Client Error: Not Found for url: http://localhost:11434/api/v1/generate
Error al con